## Agentic API 101

This document talks about the Agentic APIs in Llama Stack.

Starting Llama 3.1 you can build agentic applications capable of:

- breaking a task down and performing multi-step reasoning.
- using tools to perform some actions
  - built-in: the model has built-in knowledge of tools like search or code interpreter
  - zero-shot: the model can learn to call tools using previously unseen, in-context tool definitions
- providing system level safety protections using models like Llama Guard.

An agentic app requires a few components:
- ability to run inference on the underlying Llama series of models
- ability to run safety checks using the Llama Guard series of models
- ability to execute tools, including a code execution environment, and loop using the model's multi-step reasoning process

All of these components are now offered by a single Llama Stack Distribution. Llama Stack defines and standardizes these components and many others that are needed to make building Generative AI applications smoother. Various implementations of these APIs are then assembled together via a **Llama Stack Distribution**.

### Run Agent example

To run an agent app, check out examples demo scripts with client SDKs to talk with the Llama Stack server in our llama-stack-apps repo. With the server running, we can use the following code to run a simple agent example:

In [2]:


import os
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig


async def agent_example():
    client = LlamaStackClient(base_url="http://localhost:5000")

    agent_config = AgentConfig(
        model="Llama3.1-8B-Instruct",
        instructions="You are a helpful assistant",
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=[
            {
                "type": "brave_search",
                "engine": "brave",
                "api_key": os.getenv("BRAVE_SEARCH_API_KEY"),
            }
        ],
        tool_choice="auto",
        tool_prompt_format="function_tag",
        input_shields=[],
        output_shields=[],
        enable_session_persistence=False,
    )

    agent = Agent(client, agent_config)
    session_id = agent.create_session("test-session")
    print(f"Created session_id={session_id} for Agent({agent.agent_id})")

    user_prompts = [
        "I am planning a trip to Switzerland, what are the top 3 places to visit?",
        "What is so special about #1?",
        "What other countries should I consider to club?",
        "What is the capital of France?",
    ]

    for prompt in user_prompts:
        response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
        )

        async for log in EventLogger().log(response):
            log.print()


await agent_example()

Created session_id=b0267340-a91b-4045-9eff-606c90df058a for Agent(fd76d67a-71ef-4018-ba59-f16119004c2f)
inference> Switzerland is a beautiful country with a rich history, stunning landscapes, and vibrant culture. Here are the top 3 places to visit in Switzerland:

1. **Jungfraujoch**: Also known as the "Top of Europe," Jungfraujoch is a mountain peak located in the Swiss Alps. It's the highest train station in Europe, offering breathtaking views of the surrounding mountains, glaciers, and valleys. You can take a scenic train ride to the top and enjoy the stunning vistas, as well as visit the Ice Palace, Snow Plateau, and other attractions.
2. **Lake Geneva (Lac Léman)**: Located in the western part of Switzerland, Lake Geneva is a picturesque lake that offers stunning views of the surrounding mountains and vineyards. You can take a boat tour of the lake, visit the charming towns of Geneva and Lausanne, and enjoy the local cuisine and wine.
3. **Interlaken**: Located in the heart of the